# Goal: Develop Kalman filter and smoother using Cython and compare to numpy implementation 

In [2]:
import Cython
import cython
import scipy
import pandas as pd
import numpy as np

%load_ext cython

In [16]:
%%cython

cimport numpy as np
import numpy as np
import cython
from scipy.linalg.cython_blas cimport dgemm, ddot, dgemv, daxpy, dcopy
from libcpp cimport bool

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef void c_vcopy(double[::1] x, double[::1] y, int N, int incX=1, int incY=1) nogil:
    """
    
    custom copy
        
        arguments
            n: vectors x and y, copies x into y
        
        returns: void
        
    """
    dcopy(&N, &x[0], &incX, &y[0], &incY)

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double cddot(int N, double[::1] x, int incX, double[::1] y, int incY) nogil:
    """
    
    custom dot product
        
        arguments
            n: length of the vectors x and y
        
        returns: type double
        
    """
    return ddot(&N, &x[0], &incX, &y[0], &incY)

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef void cAx(double[::1, :] A, double[::1] x, double[::1] y, int incX=1, int incY=1, double alpha=1.0, double beta=0.0) nogil:
    """
    
    custom matrix vector alpha * Ax + beta y
    
        output: stored in y
        
    """
    
    cdef int M = A.shape[0]
    cdef int N = A.shape[1]
    
    # how many bits of memory are between A[i, j] and A[i, j + 1] -> the number of rows since A is fortran contigious
    cdef int LDA = M
    
    
    dgemv('N', &M, &N, &alpha, &A[0, 0], &LDA, &x[0], &incX, &beta, &y[0], &incY)
    

    
@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double c_xAx(double[::1, :] A, double[::1] x, double[::1] Ax, double[::1] xAx, int incX=1, int inc_Ax=1, double alpha=1.0, double beta=0.0) nogil:
    """
    
    custom matrix vector x'Ax
    
        output: xAx
        
        returns: type double (can do arithmetic in C)
        
    """
    
    cdef int M = A.shape[0]
    cdef int N = A.shape[1]
    
    # how many bits of memory are between A[i, j] and A[i, j + 1] -> the number of rows since A is fortran contigious
    cdef int LDA = M
    
    
    dgemv('N', &M, &N, &alpha, &A[0, 0], &LDA, &x[0], &incX, &beta, &Ax[0], &inc_Ax)
    return ddot(&M, &x[0], &incX, &Ax[0], &inc_Ax)


@cython.boundscheck(False)
@cython.wraparound(False)
cpdef void c_mm(double[::1, :] M1, double[::1, :] M2, double[::1, :] M3) nogil:
    """
    
    Matrix matrix multiplication AB
    
        output: matrix product
        
        returns: void, but result stored in M3
        
    """
   
    cdef int M = M1.shape[0]
    cdef int K = M1.shape[1]
    cdef int N = M2.shape[1]
    
    with gil:
        if K != M2.shape[0]:
            raise ValueError('dimension mismatch')
    
    cdef double alpha = 1.0
    cdef double beta = 0.0
    
    # Q: LDA, how many elements do you need to jump over to go from A[i,j] -> A[i, j+1] if A column major
    # A: Exactly the number of rows in A
    
    dgemm('N', 'N', &M, &N, &K, &alpha, &M1[0,0], &M,
                                             &M2[0,0], &K, &beta,
                                             &M3[0,0], &M,)

    
@cython.boundscheck(False)
@cython.wraparound(False)
cpdef void c_mm_transpose(double[::1, :] M1, double[::1, :] M2, double[::1, :] M3) nogil:
    """
    
    Matrix matrix_transpose multiplication AB'
    
        output: matrix product
        
        returns: void, but result stored in M3
        
    """
    
    cdef int M = M1.shape[0]
    cdef int K = M1.shape[1]
    cdef int N = M2.shape[1]
    
    with gil:
        if K != M2.shape[0]:
            raise ValueError('dimension mismatch')
    
    cdef double alpha = 1.0
    cdef double beta = 0.0
    
    # Q: LDA, how many elements do you need to jump over to go from A[i,j] -> A[i, j+1] if A column major
    # A: Exactly the number of rows in A
    
    dgemm('N', 'T', &M, &N, &K, &alpha, &M1[0,0], &M,
                                        &M2[0,0], &N, &beta,
                                        &M3[0,0], &M,)
    
    
@cython.boundscheck(False)
@cython.wraparound(False)
cdef void c_kalman_iteration(double[::1, :] T, double[::1, :] Z, double[::1, :] R,
                              double[::1, :] Q, double[::1, :] H, double[::1] a, 
                              double[::1, :] P, double[::1] y, 
                              double[::1] v, double[::1] M, int p, int s) nogil:
    """
    
    Kalman iteration
    
        output: matrix product
        
        returns: void, but result stored in M3
        
    """

    ### compute v
    
    # copy y into v, this is where v will be stored 
    c_vcopy(y, v, s)
    
    # v -> v - Za = y + alpha * Za because of copy line above
    with gil:
        cAx(Z, a, v ,alpha=-1)
    
    
@cython.boundscheck(False)
@cython.wraparound(False)
cpdef void c_kalman_filter(double[::1, :] T, double[::1, :] Z, double[::1, :] R,
                              double[::1, :] Q, double[::1, :] H, double[::1, :] a, 
                              double[::1, :, :] P, double[::1, :] y, 
                           double[::1, :] v, double[::1, :] M, int p, int s, int n):
    """
    
    Kalman filter
    
        output: matrix product
        
        returns: void, but result stored in M3
        
    """
    
    # testing, when assigning atemp to a, and adding 1 to a temp, will a be 1,2,3...?
    
    cdef double[::1] a_temp = np.ones((2,1), order='f')

    for i in range(n):
        
        # a[i,:], which is c contigious, is assigned a pointer to a_temp, which is fortran contigious
        a[i, :] = a_temp
        
        #c_kalman_iteration(T, Z, R, Q, H, a_temp, 
        #                      P[i, :, :], y[i, :], v[i, :], M[i, :], p, s) 
        


Error compiling Cython file:
------------------------------------------------------------
...
        
        # a[i,:], which is c contigious, is assigned a pointer to a_temp, which is fortran contigious
        a[i, :] = a_temp
        
        c_kalman_iteration(T, Z, R, Q, H, a_temp, 
                              P[::i, :, :], y[i, :], v[i, :], M[i, :], p, s) 
                              ^
------------------------------------------------------------

C:\Users\NielsOta\.ipython\cython\_cython_magic_e7a1c2dd57ac886aea16b491133eef7f.pyx:198:31: Memoryview 'double[:, :, :]' not conformable to memoryview 'double[::1, :]'.

Error compiling Cython file:
------------------------------------------------------------
...
        
        # a[i,:], which is c contigious, is assigned a pointer to a_temp, which is fortran contigious
        a[i, :] = a_temp
        
        c_kalman_iteration(T, Z, R, Q, H, a_temp, 
                              P[::i, :, :], y[i, :], v[i, :], M[i, :], p, s)